In [2]:
import socket
import matplotlib.pyplot as plt
import time
import numpy as np
import sounddevice as sd
from scipy.signal import blackman
import csv

In [3]:
import csv
from datetime import datetime

def save_waveform_data(timestamps, data, base_file_name="waveform_data"):
    """Save the waveform data to a CSV file with a name based on the current date and time.

    Parameters:
    - timestamps: A list of timestamp values.
    - data: A list of amplifier data values.
    - base_file_name: The base name of the file to save the data to, without extension.
    """
    # Get the current date and time, formatted as 'YYYY-MM-DD_HH-MM-SS'
    current_time_str = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    # Create the file name by appending the current time to the base file name
    file_name = f"{base_file_name}_{current_time_str}.csv"
    
    with open(file_name, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Timestamp (s)', 'Voltage (uV)'])  # Write header
        for timestamp, voltage in zip(timestamps, data):
            writer.writerow([timestamp, voltage])
    
    print(f'Data successfully saved to {file_name}')

# Example usage
# Assuming amplifierTimestamps and amplifierData contain your data
# save_waveform_data(amplifierTimestamps, amplifierData)



In [4]:
def generate_sound(sel):
    """Generate a sound based on selection."""
    t = np.linspace(0, DURATION, int(Fs * DURATION), endpoint=False)
    if sel == 1:  # Sine wave
        sound = np.sin(2 * np.pi * 440 * t)
    elif sel == 2:  # Modified sine wave (example)
        sound = np.sin(2 * np.pi * 880 * t)
    elif sel == 3:  # White noise
        sound = np.random.uniform(-1, 1, Fs)
    elif sel == 4:  # Silence
        sound = np.zeros(Fs)
    else:
        raise ValueError("Selection must be between 1 and 4.")
    # Ensure sound is stereo by duplicating the channel
    sound_stereo = np.column_stack((sound, sound))
    return sound_stereo

def play_sound_sel(sel, sound_events, elapsed_time, Fs=44100):
    """Play the selected sound."""
    sound = generate_sound(sel)  # Ensure this function exists and generates the correct sound based on 'sel'
    sound_events.append({'time': elapsed_time, 'sound': sel})
    sd.play(sound, Fs)
    sd.wait()
    
    


def save_sound_events(sound_events, filename='sound_events.csv'):
    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Time (s)', 'Sound Selection'])
        for event in sound_events:
            writer.writerow([event['time'], event['sound']])



In [5]:
#! /bin/env python3
# Adrian Foy September 2023

"""Example demonstrating reading of 1 second waveform data (wideband amplifier
data on channel A-010) using TCP command socket to control RHX software and TCP
waveform socket to read amplifier data.

In order to run this example script successfully, the Intan RHX software
should first be started, and through Network -> Remote TCP Control.

Command Output should open a connection at 127.0.0.1, Port 5000.
Status should read "Pending".

Waveform Output (in the Data Output tab) should open a connection at 127.0.0.1,
Port 5001. Status should read "Pending" for the Waveform Port (Spike Port is
unused for this example, and can be left disconnected).

Once these ports are opened, this script can be run to acquire ~1 second of
wideband data from channel A-010, which can then be plotted assuming
"matplotlib" is installed.
"""

import time
import socket
import numpy as np
# In order to plot the data, 'matplotlib' is required.
# If plotting is not needed, calls to plt can be removed and the data
# will still be present within the ReadWaveformDataDemo() function.
# 'matplotlib' can be installed with the command 'pip install matplotlib'
import matplotlib.pyplot as plt


def readUint32(array, arrayIndex):
    """Reads 4 bytes from array as unsigned 32-bit integer.
    """
    variableBytes = array[arrayIndex: arrayIndex + 4]
    variable = int.from_bytes(variableBytes, byteorder='little', signed=False)
    arrayIndex = arrayIndex + 4
    return variable, arrayIndex


def readInt32(array, arrayIndex):
    """Reads 4 bytes from array as signed 32-bit integer.
    """
    variableBytes = array[arrayIndex: arrayIndex + 4]
    variable = int.from_bytes(variableBytes, byteorder='little', signed=True)
    arrayIndex = arrayIndex + 4
    return variable, arrayIndex


def readUint16(array, arrayIndex):
    """Reads 2 bytes from array as unsigned 16-bit integer.
    """
    variableBytes = array[arrayIndex: arrayIndex + 2]
    variable = int.from_bytes(variableBytes, byteorder='little', signed=False)
    arrayIndex = arrayIndex + 2
    return variable, arrayIndex


In [6]:


def ReadWaveformDataDemo(sound_selection, repeats, interval, start_delay=10):
    
    """Read Waveform Data Demo.

    Uses TCP to control RHX software and read 1 second of waveform data,
    as a demonstration of TCP control and TCP data streaming, both of which
    are described in 'IntanRHX_TCPDocumentation.pdf'
    """

    # Connect to TCP command server - default home IP address at port 5000.
    print('Connecting to TCP command server...')
    scommand = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    scommand.connect(('127.0.0.1', 5000))

    # Connect to TCP waveform server - default home IP address at port 5001.
    print('Connecting to TCP waveform server...')
    swaveform = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    swaveform.connect(('127.0.0.1', 5001))

    # Query runmode from RHX software.
    scommand.sendall(b'get runmode')
    commandReturn = str(scommand.recv(COMMAND_BUFFER_SIZE), "utf-8")

    # If controller is running, stop it.
    if commandReturn != "Return: RunMode Stop":
        scommand.sendall(b'set runmode stop')
        # Allow time for RHX software to accept this command before the next.
        time.sleep(0.1)

    # Query sample rate from RHX software.
    scommand.sendall(b'get sampleratehertz')
    commandReturn = str(scommand.recv(COMMAND_BUFFER_SIZE), "utf-8")
    expectedReturnString = "Return: SampleRateHertz "
    # Look for "Return: SampleRateHertz N" where N is the sample rate.
    if commandReturn.find(expectedReturnString) == -1:
        raise GetSampleRateFailure(
            'Unable to get sample rate from server.'
        )

    # Calculate timestep from sample rate.
    timestep = 1 / float(commandReturn[len(expectedReturnString):])

    # Clear TCP data output to ensure no TCP channels are enabled.
    scommand.sendall(b'execute clearalldataoutputs')
    time.sleep(0.1)

    #OLD CODE  # Send TCP commands to set up TCP Data Output Enabled for wide
    # band of channel A-010.
    scommand.sendall(b'set a-010.tcpdataoutputenabled true')
    time.sleep(0.1)

    # Update calculations for 30 seconds of data
    # If sample rate is X Hz, then for 30 seconds, the frames would be 30*X.
    # You need to adjust the calculation of NumBlocks accordingly.

    #FRAMES_PER_BLOCK = 128  # Assuming this is defined and constant
    waveformBytesPerFrame = 4 + 2  # 4 bytes for timestamp, 2 bytes for sample
    SizeOfMagicNumber = 4
    # Calculate NumBlocks for 30 seconds of data, considering the sample rate
    sampleRate = float(commandReturn[len(expectedReturnString):])  # Extracted from the command return
    NumFrames = 30 * sampleRate  # Total frames in 30 seconds
    NumBlocks = int((NumFrames + FRAMES_PER_BLOCK - 1) / FRAMES_PER_BLOCK)  # Round up to the nearest whole block
    waveformBytesPerBlock = FRAMES_PER_BLOCK * waveformBytesPerFrame + SizeOfMagicNumber
    # Assuming each block starts with a magic number and you know the exact size of each frame/block
    # try:
    #     rawData = swaveform.recv(WAVEFORM_BUFFER_SIZE)
    #     totalBytesReceived = len(rawData)
    #     print(f"Total bytes received: {totalBytesReceived}")
        
    #     # Dynamically process rawData as it's not guaranteed to be a perfect multiple of waveformBytesPerBlock
    #     rawIndex = 0
    #     while rawIndex < totalBytesReceived:
    #         # Process each block. This is a simplified example; adapt it based on your data format.
    #         if rawIndex + waveformBytesPerBlock <= totalBytesReceived:
    #             # Read and process a block
    #             # Example: Extract timestamp and sample data from the block
    #             # Update rawIndex to the start of the next block
    #             rawIndex += waveformBytesPerBlock
    #         else:
    #             print("Incomplete block received at the end or data format mismatch.")
    #             break
    # except InvalidReceivedDataSize as e:
    #     print(f"Error processing received data: {e}")
    
    # Start recording (data collection)
    print('Starting data collection...')
    scommand.sendall(b'set runmode run')
    sound_events = []  # Initialize list to record sound playback events
    time.sleep(start_delay)  # Wait for start_delay seconds before playing sounds

    # Now start the sound playback logic
    start_playback_time = time.time()
    for i in range(repeats):
        elapsed_time = time.time() - start_playback_time
        play_sound_sel(sound_selection, sound_events, elapsed_time)
        time.sleep(interval)
    # Stop recording after the final sound playback and interval wait
    scommand.sendall(b'set runmode stop')


    # Read waveform data
    rawData = swaveform.recv(WAVEFORM_BUFFER_SIZE)
    if len(rawData) % waveformBytesPerBlock != 0:
        raise InvalidReceivedDataSize(
            'An unexpected amount of data arrived that is not an integer '
            'multiple of the expected data size per block.'
        )
    numBlocks = int(len(rawData) / waveformBytesPerBlock)

    # Index used to read the raw data that came in through the TCP socket.
    rawIndex = 0

    # List used to contain scaled timestamp values in seconds.
    amplifierTimestamps = []

    # List used to contain scaled amplifier data in microVolts.
    amplifierData = []

    for _ in range(numBlocks):
        # Expect 4 bytes to be TCP Magic Number as uint32.
        # If not what's expected, raise an exception.
        magicNumber, rawIndex = readUint32(rawData, rawIndex)
        if magicNumber != 0x2ef07a08:
            raise InvalidMagicNumber('Error... magic number incorrect')

        # Each block should contain 128 frames of data - process each
        # of these one-by-one
        for _ in range(FRAMES_PER_BLOCK):
            # Expect 4 bytes to be timestamp as int32.
            rawTimestamp, rawIndex = readInt32(rawData, rawIndex)

            # Multiply by 'timestep' to convert timestamp to seconds
            amplifierTimestamps.append(rawTimestamp * timestep)

            # Expect 2 bytes of wideband data.
            rawSample, rawIndex = readUint16(rawData, rawIndex)

            # Scale this sample to convert to microVolts
            amplifierData.append(0.195 * (rawSample - 32768))

    # If using matplotlib to plot is not desired,
    # the following plot lines can be removed.
    # Data is still accessible at this point in the amplifierTimestamps
    # and amplifierData.
    plt.plot(amplifierTimestamps, amplifierData)
    plt.title('A-010 Amplifier Data')
    plt.xlabel('Time (s)')
    plt.ylabel('Voltage (uV)')
    plt.show()
    save_waveform_data(amplifierTimestamps, amplifierData)
    save_sound_events(sound_events)

    
class GetSampleRateFailure(Exception):
    """Exception returned when the TCP socket failed to yield the sample rate
    as reported by the RHX software.
    """


class InvalidReceivedDataSize(Exception):
    """Exception returned when the amount of data received on the TCP socket
    is not an integer multiple of the excepted data block size.
    """


class InvalidMagicNumber(Exception):
    """Exception returned when the first 4 bytes of a data block are not the
    expected RHX TCP magic number (0x2ef07a08).
    """



In [8]:

if __name__ == '__main__':
    # Declare buffer size for reading from TCP command socket.
    # This is the maximum number of bytes expected for 1 read. 1024 is plenty
    # for a single text command.
    # Increase if many return commands are expected.
    #COMMAND_BUFFER_SIZE = 1024

    # Declare buffer size for reading from TCP waveform socket.
    # This is the maximum number of bytes expected for 1 read.

    # There will be some TCP lag in both starting and stopping acquisition,
    # so the exact number of data blocks may vary slightly.
    # At 30 kHz with 1 channel, 1 second of wideband waveform data is
    # 181,420 byte. See 'Calculations for accurate parsing' for more details.
    # To allow for some TCP lag in stopping acquisition resulting in slightly
    # more than 1 second of data, 200000 should be a safe buffer size.
    # Increase if channels, filter bands, or acquisition time increase.
    #WAVEFORM_BUFFER_SIZE = 400000

    # RHX software is hard-coded to always handle data in blocks of 128 frames.
    # Constants
    Fs = 44100  # Sampling frequency
    DURATION = 0.5  # Duration of the sound in seconds
    COMMAND_BUFFER_SIZE = 1024 * 10 # Example size, adjust as necessary
    WAVEFORM_BUFFER_SIZE = 800000  # Adjust based on 30 seconds of data
    FRAMES_PER_BLOCK = 128
    sound_selection = 2  # Example sound selection
    repeats = 5  # Number of times to play the sound
    interval = 2  # Interval between sounds, in seconds
    ReadWaveformDataDemo(sound_selection, repeats, interval)

Connecting to TCP command server...
Connecting to TCP waveform server...
Starting data collection...


InvalidReceivedDataSize: An unexpected amount of data arrived that is not an integer multiple of the expected data size per block.